In [ ]:
import csv
from datetime import datetime, timedelta
import math
import requests
import time

# Setup for NOAA API

https://www.ncdc.noaa.gov/cdo-web/webservices/v2

In [ ]:
with open('noaa.secret') as fp:
    TOKEN = fp.read().strip()
headers = {'token': TOKEN, 'Accept': 'application/json'}

In [ ]:
# Handle pagination
def get_all(endpoint, **params):
    results = []
    failed = 0
    while True:
        try:
            r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2' + endpoint,
                             headers=headers,
                             params=dict(params,
                                         limit='1000',
                                         offset=len(results)))
            r.raise_for_status()
        except requests.HTTPError as e:
            print("failed %r" % e)
            failed += 1
            if failed == 10:
                raise
            time.sleep(2)
            continue
        failed = 0
        obj = r.json()
        results.extend(obj['results'])
        resultset = obj['metadata']['resultset']
        count = int(resultset['count'])
        if len(results) >= count:
            break
        else:
            print("{}/{}...".format(len(results), count))
        time.sleep(0.5)
    return results

# List all datasets

In [ ]:
datasets = get_all('/datasets')
for dataset in datasets:
    print('{: <12} {}'.format(dataset['id'], dataset['name']))

In [ ]:
DATASET_ID = 'GHCND'
[dataset for dataset in datasets if dataset['id'] == DATASET_ID][0]

# List all datatypes

In [ ]:
datatypes = get_all('/datatypes', datasetid=DATASET_ID)
for type_ in sorted(datatypes, key=lambda t: t['id']):
    print('{: <5} {}'.format(type_['id'], type_['name']))

In [ ]:
DATATYPE = 'AWND'
[datatype for datatype in datatypes if datatype['id'] == DATATYPE][0]

# List all locations of type 'CITY'

In [ ]:
locations = get_all('/locations', datasetid=DATASET_ID, locationcategoryid='CITY')
locations

In [ ]:
len(locations)

# Write city->best station list

Create `noaa_city_stations.csv` by finding the best station for each city (station with the longest period covered).

In [ ]:
# TODO: take 'datacoverage' key into account

In [ ]:
location_station = {}

In [ ]:
now = datetime.today()
for location in locations:
    if location['id'] in location_station:
        continue
    if len(location_station) % 20 == 0:
        print("{}/{}...".format(len(location_station), len(locations)))
    time.sleep(0.5)
    stations = get_all('/stations', datasetid=DATASET_ID, locationid=location['id'])
    # Find best coverage
    mindate = None
    minstation = None
    for station in stations:
        smax = datetime.strptime(station['maxdate'], '%Y-%m-%d')
        if smax < now - timedelta(days=2):
            # Too old
            continue
        smin = datetime.strptime(station['mindate'], '%Y-%m-%d')
        if mindate is None or mindate > smin:
            mindate = smin
            minstation = station
    location_station[location['id']] = station

In [ ]:
len(location_station)

In [ ]:
with open('discovery/noaa/datamart_noaa_discovery/noaa_city_stations.csv', 'w', newline='\n') as fp:
    writer = csv.writer(fp)
    writer.writerow(['station_id', 'station_name', 'latitude', 'longitude', 'city_id', 'city_name'])
    for location_id, station in location_station.items():
        location = [l for l in locations if l['id'] == location_id]
        if len(location) != 1:
            print("locations for %r:\n%r\n" % (location_id, location))
        location = location[0]
        writer.writerow([station['id'], station['name'], station['latitude'], station['longitude'], location['id'], location['name']])

Getting a single station for each city didn't really work, as each station has very sparse data. The "best" station would still be missing most of the dates. Instead, let's just get all cities, and use whatever stations are available when we query.

# Write cities list

In [ ]:
location_latlong = {}

In [ ]:
for location in locations:
    if location['id'] in location_latlong:
        continue
    if len(location_latlong) % 20 == 0:
        print("{}/{}...".format(len(location_latlong), len(locations)))
    time.sleep(0.5)
    stations = get_all('/stations', datasetid=DATASET_ID, locationid=location['id'])
    # Compute average latlong
    x, y, z = 0.0, 0.0, 0.0
    for station in stations:
        lat = math.radians(station['latitude'])
        long = math.radians(station['longitude'])
        x += math.sin(lat) * math.cos(long)
        y += math.sin(lat) * math.sin(long)
        z += math.cos(lat)
    lat = math.degrees(math.atan2(z, math.sqrt(x * x + y * y)))
    long = math.degrees(math.atan2(y, x))
    location_latlong[location['id']] = lat, long

In [ ]:
with open('discovery/noaa/datamart_noaa_discovery/noaa_cities.csv', 'w', newline='\n') as fp:
    writer = csv.writer(fp)
    writer.writerow(['id', 'name', 'latitude', 'longitude'])
    for location in locations:
        try:
            lat, long = location_latlong[location['id']]
        except KeyError:
            continue
        writer.writerow([location['id'], location['name'], lat, long])

# Get data for New York

In [ ]:
data = get_all('/data',
        datasetid=DATASET_ID,
        datatypeid=DATATYPE,
        locationid='CITY:US360019',
        startdate='2018-01-01',
        enddate='2018-03-31')
data